# Detailed description of software protocol
## A fully reproducible one-stop-shop for the analysis of iTRAQ/TMT data

### Folders
TODO

### Parameters
#### Folders and database search
*Folder for spectra files (files need to be mgf) and fasta database:* 
We allow the folder _IN_ and _data_ where the latter is automatically mapped to the directory from which IsoLabeledProtocol is called (when using _run.sh_ or _run.bat_). The folder should contain the spectra files and a database of protein sequences given in a FASTA file. 
Raw files (e.g. Thermo raw files) can be converted into mgf using the msconvert tool of [ProteoWizard](http://proteowizard.sourceforge.net/tools.shtml). This requires a Windows computer. Files from different MS runs should be organized in folders as illustrated in Figure 1.






![Illustration of folder structure](misc/ExperimentalDesigns.svg)
**Figure 1**: Illustration of experimental designs and folder structure.



#### Database search
*Fasta file*: Select from FASTA files in the above given folder.
The sequence headers of the file should be formatted in a [UniProt](http://uniprot.org)-like format. See more details on a suitable version of the FASTA format in the [SearchGUI documentation](https://github.com/compomics/searchgui/wiki/DatabaseHelp). Do not provide decoy sequences as they will be created automatically.

*Precursor tolerance (ppm)*: MS tolerance in parts per million

*Fragment ion tolerance (da)*: MSn tolerance in Dalton

*Number of miscleavages*

*Further fixed modifications*: Select Carbamidomethylation or None. iTRAQ and TMT labels as fixed modifications are added separately

*Further variable modifications (Hold Ctrl to select multiple)*: Select variable modifications for the database search. Modified sequences will be used in the quantification and summarization of the proteins.

*Target (protein, peptide, and PSM) FDR*: False discovery rate for the peptide identifications.

*Quantification method*: Labelling method used in the experiment (iTRAQ/TMT)

*Number of different conditions*: Number of different types of samples (e.g. time points, treatments, cell types). This parameter is needed to assign sample types to the different labeling channels.




#### Experimental design

#### Quantitative analysis



### Output files and figures


### Code structure and scripts
#### Access

#### Parameter interface

#### R scripts
